# AutoGen本地环境搭建 - Colab 和 Mac M2环境

## 1、背景

微软开源了一个AutoGen框架，专为大语言模型快速开发而设计。然而，对于没有OpenAI Key的用户，尤其是白嫖党来说，存在一些挑战。考虑到GitHub上有许多开源的大模型，因此在本地部署这些大模型成为一个有趣的想法，以便进行AutoGen的开发。有关AutoGen的更多信息，请访问官网：[AutoGen](https://microsoft.github.io/autogen/)

## 2、需要解决的问题

1. 缺乏GPU环境：普通显卡或PC无法轻松部署大模型，而Colab提供免费的T4算力，为解决这一问题提供了可能性。
2. 非专业人士如何快速部署大模型：对于不了解量化等专业知识的用户，希望能够通过简单的操作或双击即可在本地部署大模型。令人幸运的是，GitHub上已经有相关的工具可供使用。
3. 牛头能否对接马嘴：希望通过其他软件部署的大模型能够提供API接口，并且该接口的格式是否与OpenAI的一致。GitHub上同样存在相关的工具可供参考。

## 3、准备工作

1. Autogen，Autogen框架。[AutoGen官网](https://microsoft.github.io/autogen/)
2. ollama，用于部署本地大模型，计划部署mistral-7b。[ollama官网](https://ollama.ai/)
3. litellm，用于大模型协议转换，将其转换为OpenAI标准格式的API。[litellm官网](https://docs.litellm.ai/docs/)
4. 科学上网，[***](https://github.com/Aszzc/AutoGenLocal/blob/main/https_proxy_SpeedTest.py)

## 4、实践思路
1. 先尝试用ollama部署大模型，测试能否跑起来
2. 然后启动litellm进行协议转换
3. 使用AutoGen Demo进行系统功能测试，查看系统是否正常工作

## 5、其他事项
1. Colab记得切换笔记本环境为GPU环境-T4

In [ ]:
#步骤1 安装ollama并部署mistral
'''
主要命令：
1、!curl https://ollama.ai/install.sh | sh
2、!nohup ollama serve &
2、!nohup ollama run mistral &
3、!curl -X POST http://localhost:11434/api/generate -d '{"model": "mistral","prompt":"Here is a story about llamas eating grass"}'

注：在Colab中，长期在后台运行程序会造成单元格阻塞，因此需要后台运行命令，调用命令时，使用如下方式 !nohup {cmd} &  ,查看命令的输出话用 !cat nohup.out
'''

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!nohup ollama serve &

In [ ]:
!cat nohup.out

In [ ]:
!nohup ollama run mistral & #需要等一会，要下载模型，可以用!cat nohup.out来查看进度

In [ ]:
!cat nohup.out

In [ ]:
!curl -X POST http://localhost:11434/api/generate -d '{"model": "mistral","prompt":"Here is a story about llamas eating grass"}'

In [ ]:
#步骤2 安装并启动litellm
'''
主要命令
1、!pip install litellm
2、!nohup litellm --model ollama/mistral &
3、!curl http://0.0.0.0:8000/models
'''

In [ ]:
!pip install litellm
#下面的命令出错的话尝试 !pip install 'litellm[proxy]'

In [ ]:
!nohup litellm --model ollama/mistral &

In [ ]:
!cat nohup.out

In [ ]:
!curl http://0.0.0.0:8000/models

In [ ]:
#步骤3 安装并测试AutoGen能否正常工作

'''
主要命令
1、 !pip install autogen
'''

In [ ]:
!pip install pyautogen

In [ ]:
llm_config = {
    'config_list':[
        {
            'model': "ollama/mistral",
            'api_key': "api",
            'base_url':'http://127.0.0.1:8000/'
        },
    ]
}

In [ ]:
import autogen

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
)

user_proxy.initiate_chat(
    assistant,
    message="""现在是什么时间""",
)